# Electrode size affects spike shapes

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from matplotlib.patches import Ellipse
import LFPy
from brainsignals.plotting_convention import mark_subplots, simplify_axes
from brainsignals.neural_simulations import return_hay_cell
import brainsignals.neural_simulations as ns

np.random.seed(12345)
ns.load_mechs_from_folder(ns.cell_models_folder)

In [ ]:
def insert_current_stimuli(cell):
    stim_params = {'amp': -0.4,
                   'idx': 0,
                   'pptype': "ISyn",
                   'dur': 1e9,
                   'delay': 0}
    synapse = LFPy.StimIntElectrode(cell, **stim_params)
    return synapse, cell

In [ ]:
tstop = 150
# Time window to extract spike from:
t0 = 118
t1 = 123

dt = 2**-6
cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=False)
ns.point_axon_down(cell)
syn, cell = insert_current_stimuli(cell)
cell.simulate(rec_imem=True, rec_vmem=True)
t0_idx = np.argmin(np.abs(cell.tvec - t0))
t1_idx = np.argmin(np.abs(cell.tvec - t1))

cell.vmem = cell.vmem[:, t0_idx:t1_idx]
cell.imem = cell.imem[:, t0_idx:t1_idx]
cell.tvec = cell.tvec[t0_idx:t1_idx] - cell.tvec[t0_idx]

elec_radii = np.array([1, 10, 25, 50])

eaps = []

elec_params = dict(
            sigma = 0.3,      # extracellular conductivity
            x = np.array([20]),
            y = np.array([0]),
            z = np.array([0]),
            method = 'root_as_point',
        )

for elec_radius in elec_radii:
    if elec_radius > 1e-9:
        elec_params.update(
            N = np.array([[1, 0, 0]]), # surface normals
            r = elec_radius,           # contact site radius
            n = elec_radius * 10,      # datapoints for averaging
        )

    elec = LFPy.RecExtElectrode(cell, **elec_params)
    LFP = elec.get_transformation_matrix() @ cell.imem * 1000
    eaps.append(LFP[0])


In [ ]:
elec_clrs = {r: plt.cm.Reds(0.1 + i / (len(elec_radii) - 1))
             for i, r in enumerate(elec_radii)}

plt.close("all")
fig = plt.figure(figsize=[6, 3.2])
fig.subplots_adjust(left=0.14, bottom=0.18, top=0.85, right=0.98,
                    hspace=0.5)

ax_morph = fig.add_axes([0.01, 0.03, 0.7, 0.96], frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[-150, 150],
                        ylim=[-100, 100])

ax_eap = fig.add_axes([0.75, 0.57, 0.24, 0.3],
                      xlabel="time (ms)", ylabel="µV")
ax_eap_norm = fig.add_axes([0.75, 0.1, 0.24, 0.3],
                           xlabel="time (ms)", ylabel="normalized")

zips = []
for x, z in cell.get_pt3d_polygons():
    zips.append(list(zip(x, z)))
polycol = PolyCollection(zips, edgecolors='none',
                         facecolors='gray', zorder=100, rasterized=False)
ax_morph.add_collection(polycol)

lines = []
line_names = []
for i, elec_radius in enumerate(elec_radii):
    c = elec_clrs[elec_radius]

    if elec_radius > 1e-9:
        el = Ellipse((elec.x[0], elec.z[0]), elec_radius / 2, 2*elec_radius,
                 facecolor=c, clip_on=False, zorder=-i)
        ax_morph.add_artist(el)
    else:
        ax_morph.plot(elec.x[0], elec.z[0], '.', c=c, ms=3)

    l, = ax_eap.plot(cell.tvec, eaps[i], c=c)
    ax_eap_norm.plot(cell.tvec, eaps[i] / np.max(np.abs(eaps[i])), c=c)
    lines.append(l)
    line_names.append("$r$=%d µm" % elec_radius)

fig.legend(lines, line_names, loc="lower left", frameon=False, ncol=4)
mark_subplots(ax_morph, ypos=0.95, xpos=0.1)
mark_subplots([ax_eap, ax_eap_norm], "BC")
simplify_axes(fig.axes)
fig.savefig("fig_elec_size_effect.pdf")
